**1. Can you think of a few applications for a sequence-to-sequence RNN? What about a sequence-to-vector RNN? And a vector-to-sequence RNN?**

- Sequence-to-Sequence

This is useful for creative RNNs. Give it a melody, and then it will continuously generate more of the melody.

- Sequence-to-Vector

A RNN like this would be useful if you want to input a sequence of arbitrary length and then output some n-dimensional vector. For example, you could take in a movie review or dialogue script of any length, and then output either a -1 to 1 sentiment score or perhaps a larger vector of various emotions (happy, excited, angry, sad, etc.)

- Vector-to-Sequence

This is good for taking your n-dimensional vector (it could even be an image) and outputting an arbitrary-length sequence, for example, autocaptioning of images or videos.

**2. Why do people use encoder-decoder RNNs rather than plain sequence RNNs for automatic translation?**

In translating a sentence, you often need to hear every part of it. Words near the end of a sentence can affect words at the beginning. Thus, it is better to have an encoder-decoder RNN that digests the whole sequence, finds some latent representation of it, and then decodes it into another language from that representation.

**3. How could you combine a convolutional neural network with a RNN to classify videos?**

You could generate features in a smaller space from the convolutional network from each frame of the video, then take **n_step** chunks of data and pass the whole sequence of features per frame of the video into the RNN.

**4. What are the advantages of building an RNN using `dynamic_rnn()` rather than `static_rnn()`?**

This does not apply in Tensorflow v2

**5. How can you deal with variable-length input sequences? What about variable-length output sequences?**

You could pad your instances in each batch so that they all have the same length as the largest instance in that batch.

For variable-length outputs, you can define an "end-of-sequence" character, and any outputs from the RNN after that is ignored.

**6. What is a common way to distribute training and execution of a deep RNN across multiple GPUs?**

This does not apply in Tensorflow v2

**7. Embeddded Reber grammars were used by Hochreiter and Schmidhuber in their paper about LSTMs. They are artificial grammars that produce strings such as "BPBTSXXVPSEPE." Check out Jenny Orr's nice introduction (http://homl.info/108) to this topic. Choose a particular embedded Reber grammar (such as the one represented on Jenny Orr's page), then train an RNN to identify whether a string respects that grammar or not. You will first need to write a function capable of generating a training batch containing about 50% strings that respect the grammar, and 50% that don't.**

In [1]:
import numpy as np

In [5]:
def coinflip():
    return np.random.random() < .5

def p_node():
    val = 'P'
    if coinflip():
        val += 'S'
    else:
        val += t_node()
    return val

def t_node():
    val = ''
    for _ in range(np.random.randint(0,4)):
        val += 'T'
    val += 'V'
    if coinflip():
        val += 'V'
    else:
        val += p_node()
    return val

def gen_reber_string():
    val = 'B'
    if coinflip():
        val += 'T'
        for _ in range(np.random.randint(0,4)):
            val += 'S'
        val += 'X'
        if coinflip():
            val += 'X'
            val += t_node()
        else:
            val+='S'
    else:
        val += 'P'
        val += t_node()
    return val + 'E'

In [22]:
vals = [gen_reber_string() for _ in range(2000)]
print('Mean:',np.mean([len(val) for val in vals]))
print('Std:', np.std([len(val) for val in vals]))
print('First 5:', vals[:5])

Mean: 8.53
Std: 3.0081722025176685
First 5: ['BTXXVPTTVVE', 'BTSXSE', 'BPTTTVVE', 'BTSXXTTVVE', 'BPVVE']


In [30]:
def gen_non_reber_string():
    val = 'B'
    choices = 'T,P,X,S,V'.split(',')
    num_chars = np.random.randint(3,11)
    for character in np.random.choice(choices, size=num_chars):
        val += character
    return val + 'E'

In [31]:
vals = [gen_non_reber_string() for _ in range(2000)]
print('Mean:',np.mean([len(val) for val in vals]))
print('Std:', np.std([len(val) for val in vals]))
print('First 5:', vals[:5])

Mean: 8.5335
Std: 2.308219606103371
First 5: ['BSPPE', 'BPTTXE', 'BTVXSSPXVE', 'BVSPE', 'BVXSPPVPPVTE']


In [32]:
# Generating data
x, y = [], []
for _ in range(50000):
    if coinflip():
        x.append(gen_reber_string())
        y.append(1)
    else:
        x.append(gen_non_reber_string())
        y.append(0)

In [47]:
char2indx = {u:i for i, u in enumerate('BPSXVTE')}
text_as_int = []
for text in x:
    text_as_int.append([char2indx[c] for c in text])
text_as_int = np.array(text_as_int)

In [ ]:
def indx_to_char(arr):
    return []

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text_as_int,y,test_size=.2)
x_train[0]

[0, 1, 5, 5, 5, 4, 4, 6]

In [82]:
(type(x_train),type(x_test),type(y_train),type(y_test))

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [79]:
y_train = np.asarray(y_train).reshape((-1,1))
y_test = np.asarray(y_test).reshape((-1,1))

In [80]:
y_train.shape

(40000, 1)

In [35]:
import tensorflow as tf

In [62]:
rnn = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(7, 4, input_shape=(7,)),
    tf.keras.layers.GRU(100),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [63]:
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [81]:
rnn.fit(x_train, y_train, epochs=15)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).